In [ ]:
from typing import Tuple
import pandas as pd
import numpy as np

from pathlib import Path
from matplotlib import pyplot, image

In [ ]:
def returnAnnotations(file_attributos, file_annotations):
  df_attributes = pd.read_csv(file_attributos, header = None)
  list_attributes = list()
  list_attributes.append(df_attributes[0].values)
  handle = open(file_annotations)
  scores = dict()
  confidence = dict()
  for line in handle:
      words = line.split()
      i=0
      for word in words:
          if i == 0:
              scores[word]=list()
              confidence[word] = list()
              i+=1
          else:
              values = word.split(',')
              scores[words[0]].append(float(values[0]))
              confidence[words[0]].append(float(values[1]))
              i+=1
  scores = pd.DataFrame(scores).T
  scores.columns = list_attributes
  confidence = pd.DataFrame(confidence).T
  confidence.columns = list_attributes
  return scores,confidence


In [ ]:
file_attributos = '../../data/annotations/attributes.txt'
file_annotations = '../../data/annotations/annotations.tsv'
file_training = '../../data/training_test_splits/holdout_split/training.txt'
file_test = '../../data/training_test_splits/holdout_split/test.txt'
def returnTrainingTest(file_attributos, file_annotations, file_training, file_test):
  scores,confidence = returnAnnotations(file_attributos, file_annotations)
  df_training = pd.read_csv(file_training, header = None)
  df_test = pd.read_csv(file_test, header = None)
  scores_training = scores[scores.index.isin(df_training[0].values)]
  scores_test = scores[scores.index.isin(df_test[0].values)]
  confidence_training = confidence[confidence.index.isin(df_training[0].values)]
  confidence_test = confidence[confidence.index.isin(df_test[0].values)]
  return scores_training, scores_test, confidence_training, confidence_test

In [ ]:
scores_training, scores_test, confidence_training, confidence_test = returnTrainingTest(file_attributos, file_annotations, file_training, file_test)

In [ ]:
scores_training

In [ ]:
scores_test

In [ ]:
confidence_training

In [ ]:
confidence_test

In [ ]:
confidence_training = confidence_training.apply(lambda x: [0 if y <= 5 else 1 for y in x])
confidence_training

In [ ]:
confidence_test = confidence_test.apply(lambda x: [0 if y <= 5 else 1 for y in x])
confidence_test

In [ ]:
#df_training = pd.DataFrame(scores_training*confidence_training, columns=scores_training.columns, index=scores_training.index)
df_training = scores_training
df_training

In [ ]:
#df_test = pd.DataFrame(scores_test*confidence_test, columns=scores_test.columns, index=scores_test.index)
df_test = scores_test
df_test

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display_markdown
display_markdown('# Verificação visual do balanço do banco de dados no conjunto de treino.',raw=True)
plt.figure(figsize=(70, 4))
plt.title('Frequencia das classes no conjunto de treino')
palette = sns.color_palette('gist_rainbow', len(df_training.columns.values))
sns.barplot(df_training.columns.values, df_training.sum().values,palette=palette)

In [ ]:
import os
def returnPathImages():
    imageLD = {}
    path_img = Path('../../data/ignored/images/imageLD/')
    datasets = os.listdir(path_img)
    for dataset in datasets:
        imgs_name = os.listdir(path_img/Path(dataset))
        for name in imgs_name:
            img_file = path_img/Path(dataset)/Path(name)
            imageLD[str(Path(dataset)/Path(name))]=img_file
    return imageLD

In [ ]:
images_paths = returnPathImages()

In [ ]:
images_paths

In [ ]:
df_training_image = df_training[df_training.index.isin(images_paths.keys())]
df_training_image

In [ ]:
df_test_image = df_test[df_test.index.isin(images_paths.keys())]
df_test_image

In [ ]:
iter_img = iter(images_paths.items())

In [ ]:
a_name, a_img = next(iter_img)

In [ ]:
image.imread(a_img)

In [ ]:
a_name

In [ ]:
df_training.loc[a_name]

In [ ]:
images_paths_train = {k: v for k, v in images_paths.items() if k in df_training.index}

In [ ]:
images_paths_teste = {k: v for k, v in images_paths.items() if k in df_test.index}

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
import cv2

In [ ]:
n_imgs = 100
X_train = []
y_train = []
for i, (img_name, img_path) in enumerate(images_paths_train.items()):
    if i >= n_imgs: break
    x = image.imread(img_path)
    x = cv2.resize(x, (400,400))
    x = x.flatten()
    print(len(x))
    X_train.append(x)
    y_train.append(df_training.loc[img_name].values)

In [ ]:
n_imgs = 100
X_teste = []
y_teste = []
for i, (img_name, img_path) in enumerate(images_paths_teste.items()):
    if i >= n_imgs: break
    x = image.imread(img_path)
    x = cv2.resize(x, (400,400))
    x = x.flatten()
    print(len(x))
    X_teste.append(x)
    y_teste.append(df_test.loc[img_name].values)

In [ ]:
clf = MultiOutputRegressor(SVR())

In [ ]:
clf.fit(X_train, y_train)


In [ ]:
X_train

In [ ]:
y_train

In [ ]:
y_predicted = clf.predict(X_teste)

In [ ]:
y_predicted

In [ ]:
np.square(y_teste - y_predicted).mean()